In [1]:
pip install ultralytics --upgrade


  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
datase

In [2]:
pip install deep_sort_realtime

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy==1.25.2 --upgrade

  Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.25.2 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.25.2 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.25.2 which is incompatible.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.

In [ ]:
import cv2
import time
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# -----------------------------
# 1️⃣ Initialize models
# -----------------------------
model = YOLO("yolov8s.pt")  # YOLOv8 small model
tracker = DeepSort(max_age=30)  # DeepSORT tracker

# -----------------------------
# 2️⃣ Video input
# -----------------------------
video_path = "/kaggle/input/highway/highway.mp4"  # input video path
cap = cv2.VideoCapture(video_path)

# Optional: resize for speed
width, height = 640, 480

# Class labels for YOLOv8
CLASS_NAMES = model.names

# -----------------------------
# 3️⃣ Video output
# -----------------------------
output_path = "/kaggle/working/tracking_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))  # use input video FPS
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# -----------------------------
# 4️⃣ Main loop
# -----------------------------
prev_time = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame
    frame = cv2.resize(frame, (width, height))

    # YOLO detection
    results = model(frame, stream=True)

    detections = []
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = CLASS_NAMES[cls]

            if conf > 0.3:
                detections.append(([x1, y1, x2, y2], conf, label))

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracks
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        label = track.det_class if hasattr(track, "det_class") else "Unknown"

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} ID:{track_id}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Calculate FPS
    curr_time = time.time()
    fps_display = 1 / (curr_time - prev_time)
    prev_time = curr_time
    cv2.putText(frame, f"FPS: {int(fps_display)}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write frame to output video
    out.write(frame)

# -----------------------------
# 5️⃣ Cleanup
# -----------------------------
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Tracking video saved to: {output_path}")



0: 480x640 4 cars, 1 truck, 45.1ms
Speed: 1.7ms preprocess, 45.1ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 3 trucks, 6.9ms
Speed: 1.4ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 cars, 1 truck, 6.9ms
Speed: 1.5ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 2 trucks, 6.9ms
Speed: 1.6ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 2 trucks, 6.8ms
Speed: 2.0ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 cars, 3 trucks, 6.8ms
Speed: 1.6ms preprocess, 6.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 4 trucks, 6.8ms
Speed: 2.1ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 cars, 3 trucks, 6.9ms
Speed: 1.6ms preprocess, 6.9ms inference